In [35]:
import pandas as pd
from utils import data_util as du, preproc_utils as pu, extractor_util as eu
import importlib
import os

In [4]:
DATA_SRC_DIR = '/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/data'
EXT_DATA_DIR = '/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/data/external'
INT_DATA_DIR = '/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/data/internal'

In [11]:
# Usage example
dataset = du.DatasetLoader(EXT_DATA_DIR)
brandenburg = dataset.load_scene('brandenburg_gate')


In [14]:
# Access data
images_path = brandenburg.images_dir
calibration_data = brandenburg.calibration

In [ ]:
for i in images_path.iterdir():
    print(i)

In [16]:
# Usage
config = pu.PreprocessConfig(size=(512, 512), normalize=False)
preprocessor = pu.ImagePreprocessor(config)

In [66]:
# Get two images from images_path
images = list(images_path.iterdir())[:2]
image1, image2 = images[0], images[1]
print(image1, image2)

/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/data/external/train/brandenburg_gate/images/10624116_2746543073.jpg /Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/data/external/train/brandenburg_gate/images/96174448_2036661198.jpg


In [68]:
img1 = preprocessor.process_image(image1)
img2 = preprocessor.process_image(image2)

In [51]:
default_sift_config = eu.SIFTConfig()
extractor_config = eu.FeatureExtractorConfig(default_sift_config)
extractor = eu.FeatureExtractor(config=extractor_config)

In [69]:
i1f = extractor.extract_features(img1)
i2f = extractor.extract_features(img2)

In [123]:
importlib.reload(eu)

<module 'utils.extractor_util' from '/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/utils/extractor_util.py'>

In [124]:
bf_config = eu.BFMatcherConfig(knn=True)
matcher_config = eu.MatcherConfig(bf_config)

In [125]:
matcher = eu.FeatureMatcher(matcher_config)

In [126]:
matches = matcher.match_features(i1f, i2f)

In [127]:
valid, kp1, kp2 = matcher.filter_lowe_matches(matches, i1f, i2f)

In [131]:
kp1[0].pt

(12.037242889404297, 378.438232421875)